In [1]:
import sys
import yaml
from sklearn.model_selection import train_test_split

sys.path.append('../../../../')

In [2]:
from scripts.utils import load_data, preprocess_data_label_encoder, create_sequences, train_and_test, simulate_predictions, simulate_predictions_from_pretrained

No GPU available. Running on CPU.


In [3]:
from src.models.bidir_lstm_multihead import build_model

In [4]:
YAML_FILE = '../../../../configs/games/tic-tac-toe/bidir_lstm/50k.yml'

In [5]:
with open(YAML_FILE, 'r') as f:
    config = yaml.safe_load(f)
print(config)

{'name': '/games/tic-tac-toe/bidir_lstm/50k', 'model': {'type': 'Bidirectional Multihead LSTM', 'lstm_units': 128, 'num_lstm_layers': 2, 'attention': True}, 'training': {'batch_size': 64, 'learning_rate': 0.001, 'num_epochs': 5}, 'data': {'game': 'tic-tac-toe', 'sequence_length': 20, 'max_event_length': 50, 'path': '/games/tic-tac-toe/50k_single_agent.csv'}}


In [6]:
model_params = {
    'lstm_units': config['model']['lstm_units'],
    'num_lstm_layers': config['model']['num_lstm_layers'],
    'attention': config['model']['attention']
}

In [7]:
# Load and preprocess data
df = load_data(f'../../../../data/processed/{config['data']['path']}')
df, e_event, e_agent, e_context = preprocess_data_label_encoder(df)

# Create sequences
sequence_length = config['data']['sequence_length']
X, y = create_sequences(df, sequence_length)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
model = train_and_test(
    X_train, 
    X_test, 
    y_train, 
    y_test, 
    e_event, 
    e_agent, 
    e_context, 
    name=config['name'], 
    epochs=config['training']['num_epochs'], 
    batch_size=config['training']['batch_size'],
    model_params=model_params,
    build_model=build_model,
)

Epoch 1/5
6017/6017 ━━━━━━━━━━━━━━━━━━━━ 255s 42ms/step - loss: 2.4377 - output_1_accuracy: 0.9208 - output_2_accuracy: 0.8616 - output_3_accuracy: 0.2535 - val_loss: 2.0888 - val_output_1_accuracy: 0.9325 - val_output_2_accuracy: 0.9325 - val_output_3_accuracy: 0.2916
Epoch 2/5
6017/6017 ━━━━━━━━━━━━━━━━━━━━ 241s 40ms/step - loss: 2.0443 - output_1_accuracy: 0.9320 - output_2_accuracy: 0.9317 - output_3_accuracy: 0.3019 - val_loss: 1.9004 - val_output_1_accuracy: 0.9323 - val_output_2_accuracy: 0.9325 - val_output_3_accuracy: 0.3224
Epoch 3/5
6017/6017 ━━━━━━━━━━━━━━━━━━━━ 236s 39ms/step - loss: 1.8908 - output_1_accuracy: 0.9344 - output_2_accuracy: 0.9341 - output_3_accuracy: 0.3199 - val_loss: 1.7453 - val_output_1_accuracy: 0.9446 - val_output_2_accuracy: 0.9442 - val_output_3_accuracy: 0.3379
Epoch 4/5
6017/6017 ━━━━━━━━━━━━━━━━━━━━ 296s 49ms/step - loss: 1.7075 - output_1_accuracy: 0.9472 - output_2_accuracy: 0.9472 - output_3_accuracy: 0.3421 - val_loss: 1.6754 - val_output_1_a

Test Results:
loss: 1.5630
compile_metrics: 0.9633


In [9]:
simulate_predictions(
    data=X,
    model=model,
    e_event=e_event,
    e_agent=e_agent,
    e_context=e_context,
    n = 15,
    k = config['data']['sequence_length']
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 507ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Initial sequence:
MOVE, O, 2,0
GAME_END, system, O
GAME_START, system, New Game
MOVE, X, 2,0
MOVE, O, 1,1
MOVE, X, 2,1
MOVE, O, 1,0
MOVE, X, 0,0
MOVE, O, 0,2
MOVE, X, 1,2
MOVE, O, 2,2
MOVE, X, 0,1
GAME_END, system, draw
GAME_START, system, New Game
MOVE, X, 0,2
MOVE, O, 2,2
MOVE, X, 1,0
MOVE, O, 2,1
MOVE, X, 0,0
MOVE, O, 2,0

Predicted moves:
Move 1: GAME_END, system, O
Move 2: GAME_START, system, New Game
Move 3: MOVE, X, 2,

In [10]:
simulate_predictions_from_pretrained(
    data=X,
    modelpath=f'../../../../models/{config['name']}.h5',
    e_event=e_event,
    e_agent=e_agent,
    e_context=e_context,
    n = 15,
    k = config['data']['sequence_length']
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 510ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Initial sequence:
MOVE, O, 1,0
MOVE, X, 2,2
GAME_END, system, X
GAME_START, system, New Game
MOVE, X, 0,1
MOVE, O, 2,1
MOVE, X, 0,2
MOVE, O, 1,0
MOVE, X, 1,1
MOVE, O, 0,0
MOVE, X, 2,0
GAME_END, system, X
GAME_START, system, New Game
MOVE, X, 1,1
MOVE, O, 0,0
MOVE, X, 2,0
MOVE, O, 1,2
MOVE, X, 2,1
MOVE, O, 0,2
MOVE, X, 0,1

Predicted moves:
Move 1: MOVE, O, X
Move 2: MOVE, X, 1,0
Move 3: GAME_END, system, X
Move 4: GAME_START,